# 卷积神经网络实践：混凝土裂缝图像分类

在这个实践中，我们将学习如何使用PyTorch实现卷积神经网络(CNN)模型。我们将以混凝土裂缝图像分类为例，这是土木工程中的一个重要应用。

## 1. 导入必要的库

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import os

## 2. 准备数据

通常，我们会使用真实的混凝土裂缝图像数据集。但为了演示目的，我们将创建一个简单的模拟数据集。在实际应用中，您需要替换这部分代码以加载和处理真实的图像数据。

In [ ]:
def create_dummy_image(has_crack):
    img = np.zeros((64, 64), dtype=np.uint8)
    if has_crack:
        # 添加裂缝
        start = np.random.randint(0, 32)
        img[start:start+32, 32] = 255
    return Image.fromarray(img)

# 创建模拟数据集
n_samples = 1000
images = []
labels = []

for i in range(n_samples):
    has_crack = i % 2 == 0
    img = create_dummy_image(has_crack)
    images.append(img)
    labels.append(1 if has_crack else 0)

# 显示一些样本图像
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title(f"Label: {'Crack' if labels[i] == 1 else 'No Crack'}")
    ax.axis('off')
plt.tight_layout()
plt.show()

## 3. 数据预处理和加载

In [ ]:
class CrackDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# 定义数据转换
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 创建数据集
train_dataset = CrackDataset(X_train, y_train, transform=transform)
test_dataset = CrackDataset(X_test, y_test, transform=transform)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"训练集大小: {len(train_dataset)}")
print(f"测试集大小: {len(test_dataset)}")

## 4. 定义卷积神经网络模型

In [ ]:
class CrackCNN(nn.Module):
    def __init__(self):
        super(CrackCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 2)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 16 * 16)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 实例化模型
model = CrackCNN()
print(model)

## 5. 定义损失函数和优化器

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## 6. 训练模型

In [ ]:
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

print('Finished Training')

## 7. 评估模型

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'测试集准确率: {accuracy:.2f}%')

## 8. 可视化一些预测结果

In [ ]:
def imshow(img, ax):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    ax.imshow(npimg, cmap='gray')

model.eval()
images, labels = next(iter(test_loader))

outputs = model(images.to(device))
_, predicted = torch.max(outputs, 1)

fig, axes = plt.subplots(2, 5, figsize=(15, 6))
for i, ax in enumerate(axes.flat):
    imshow(images[i].squeeze(), ax)
    # set font to red if prediction is incorrect
    ax.set_title(f"Pred: {'Crack' if predicted[i] == 1 else 'No Crack'}\nTrue: {'Crack' if labels[i] == 1 else 'No Crack'}", color='red' if predicted[i] != labels[i] else 'black')
    ax.axis('off')
plt.tight_layout()
plt.show()

## 9. 总结

在这个实践中，我们学习了如何使用PyTorch实现一个简单的卷积神经网络来进行混凝土裂缝图像分类。主要步骤包括:

1. 准备和可视化数据
2. 数据预处理和加载
3. 定义CNN模型结构
4. 选择损失函数和优化器
5. 训练模型
6. 评估模型性能
7. 可视化预测结果

这个例子展示了如何将深度学习应用于土木工程中的实际问题。通过这种方法，我们可以自动检测混凝土结构中的裂缝，这对于结构健康监测和维护决策都有重要意义。

在实际应用中，您需要使用更大的真实数据集，可能还需要进行数据增强和更复杂的模型设计。此外，考虑到混凝土裂缝的复杂性，您可能还需要进行裂缝定位和严重程度评估等更高级的任务。